#### CNN Model:

In [1]:
# libraries
import pandas as pd

import numpy as np
from numpy import mean
from numpy import std
from tensorflow import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.callbacks import EarlyStopping

from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report)

import plotly.figure_factory as ff

import os
import scipy.io as sio
from scipy.fft import fft

c:\Users\carlo\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load the dataset, returns train, test and validation X and y elements
def load_dataset():

	# load all data
	with open('data/processed/trainX.npy', 'rb') as f:
		trainX = np.load(f)
	with open('data/processed/trainy.npy', 'rb') as f:
		trainy = np.load(f)
	with open('data/processed/testX.npy', 'rb') as f:
		testX = np.load(f)
	with open('data/processed/testy.npy', 'rb') as f:
		testy = np.load(f)
	with open('data/processed/valX.npy', 'rb') as f:
		valX = np.load(f)
	with open('data/processed/valy.npy', 'rb') as f:
		valy = np.load(f)
	
	print("train: ",trainX.shape, trainy.shape, "\ntest: ", testX.shape, testy.shape, "\nval: ", valX.shape, valy.shape)
	
	return trainX, trainy, testX, testy, valX, valy

In [3]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, valX, valy):

    verbose, epochs, batch_size = 0, 50, 32

    # Define early stopping criteria
    early_stop = EarlyStopping(monitor='val_accuracy', patience=5, mode='max')
    
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    model = Sequential()
    model.add(Conv1D(filters=3, kernel_size=5, activation='relu',input_shape=(n_timesteps,n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=12, kernel_size=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=(valX, valy), callbacks=[early_stop])
    
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    pred_train = model.predict(trainX, verbose=0)
    pred_test = model.predict(testX, verbose=0)
    
    # save model
    models_dir = 'models/'
    existing_models = [filename for filename in os.listdir(models_dir) if filename.startswith('cnn_model')]
    num_model = len(existing_models)+1
    filename = f'models/cnn_model_{num_model}.h5'
    model.save(filename)

    return accuracy, pred_train, pred_test

In [4]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [7]:
trainX, trainy, testX, testy, valX, valy = load_dataset()

MemoryError: Unable to allocate 82.4 MiB for an array with shape (10798080,) and data type float64

In [9]:
pd.DataFrame(np.argmax(trainy,axis=1)).value_counts()

1    16899
0    16885
2    15425
dtype: int64

In [10]:
pd.DataFrame(np.argmax(testy,axis=1)).value_counts()

1    3600
0    3552
2    3393
dtype: int64

In [11]:
pd.DataFrame(np.argmax(valy,axis=1)).value_counts()

0    3617
1    3594
2    3334
dtype: int64

In [5]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy, valX, valy = load_dataset()
	# repeat experiment
	scores = list()
	train_accs = list()
	test_accs = list()
	for r in range(repeats):
		score, pred_train, pred_test = evaluate_model(trainX, trainy, testX, testy, valX, valy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
		train_acc = accuracy_score(np.argmax(trainy,axis=1), np.argmax(pred_train,axis=1))*100
		test_acc = accuracy_score(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1))*100
		train_accs.append(train_acc)
		test_accs.append(test_acc)

	# summarize results
	summarize_results(scores)
	print('Train accuracy: ')
	summarize_results(train_accs)
	print('Test accuracy: ')
	summarize_results(test_accs)


In [61]:
# run the experiment
run_experiment(10)

train:  (49209, 512, 2) (49209, 3) 
test:  (10545, 512, 2) (10545, 3) 
val:  (10545, 512, 2) (10545, 3)
>#1: 92.926
>#2: 91.939
>#3: 88.174
>#4: 91.636
>#5: 92.878
>#6: 92.395
>#7: 90.507
>#8: 92.338
>#9: 87.606
>#10: 83.822
[92.92555451393127, 91.93930625915527, 88.1744921207428, 91.6358470916748, 92.87813901901245, 92.39450097084045, 90.50735235214233, 92.33760237693787, 87.60550022125244, 83.82171392440796]
Accuracy: 90.422% (+/-2.832)
Train accuracy: 
[94.57213111422708, 94.73267085289277, 92.20264585746509, 97.8682761283505, 97.29724237436241, 95.75077729683595, 95.4805015342722, 94.28153386575626, 88.6341929321872, 83.84645085248633]
Accuracy: 93.467% (+/-4.053)
Test accuracy: 
[92.92555713608344, 91.93930772878142, 88.17449027975344, 91.63584637268848, 92.87814129919393, 92.39449976292082, 90.50734945471788, 92.3376007586534, 87.60550023707918, 83.82171645329541]
Accuracy: 90.422% (+/-2.832)


In [66]:
# load data
trainX, trainy, testX, testy, valX, valy = load_dataset()

score, pred_train, pred_test = evaluate_model(trainX, trainy, testX, testy, valX, valy)
score = score * 100.0
train_acc = accuracy_score(np.argmax(trainy,axis=1), np.argmax(pred_train,axis=1))
test_acc = accuracy_score(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1))
print('Train accuracy: ',train_acc)
print('Test accuracy: ',test_acc)

train:  (49209, 512, 2) (49209, 3) 
test:  (10545, 512, 2) (10545, 3) 
val:  (10545, 512, 2) (10545, 3)
Train accuracy:  0.9338535633725538
Test accuracy:  0.9012802275960171


In [67]:
print(classification_report(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1), target_names=['Healthy', 'OR fault', 'IR fault'],digits=4))

              precision    recall  f1-score   support

     Healthy     0.9233    0.9017    0.9124      3552
    OR fault     0.8998    0.9158    0.9078      3600
    IR fault     0.8804    0.8854    0.8829      3393

    accuracy                         0.9013     10545
   macro avg     0.9012    0.9010    0.9010     10545
weighted avg     0.9015    0.9013    0.9013     10545



In [11]:
# testing
# Construimos una visualización para la matriz de confusión
z_test = confusion_matrix(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1))
# Reformateo la matriz para que me quede mejor el gráfico
z_test[[0,2],:] = z_test[[2,0],:]
x = ['Healthy', 'OR fault', 'IR fault']
y = ['IR fault', 'OR fault', 'Healthy']
z_text = [[str(y) for y in x] for x in z_test]
heatmap = ff.create_annotated_heatmap(z_test, x=x, y=y, annotation_text=z_text, colorscale='tealrose')
heatmap.update_layout(title_text='Testing',height=300,width=600,
                      xaxis_title="Predicted Label",yaxis_title="True Label")
heatmap.show()

Check if it is making more mistakes for one specific speed setting.

In [69]:
# setting
indicesMistakes = np.where((np.argmax(testy,axis=1) != np.argmax(pred_test,axis=1)))[0]
testX_mistakes = testX[indicesMistakes]
pd.DataFrame(testX_mistakes[:,0,1]).value_counts()

# Mistakes are quite balanced between samples with 900 and 1500 rpm for rotational speed
# but the testX only had 1/4 of data of 900 rpm
# so the proportion is bigger

900.0     556
1500.0    485
dtype: int64